In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
# Librerias
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
# Carga de datos
from google.colab import drive
drive.mount("/content/drive")
file_path = "/content/drive/MyDrive/TP 1 - ORGANIZACIÓN DE DATOS/googleplaystore.csv"
reviews_file_path = "/content/drive/MyDrive/TP 1 - ORGANIZACIÓN DE DATOS/googleplaystore_user_reviews.csv"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
google_play_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv(file_path, header=True, inferSchema=True)
user_reviews_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv(reviews_file_path, header=True, inferSchema=True)
google_play_data = google_play_data.rdd
user_reviews_data = user_reviews_data.rdd
# Filtra las filas en las que la categoría no sea "1.9" y crea un nuevo RDD sin esa fila
google_play_data = google_play_data.filter(lambda x: x['Category'] != "1.9")


# 14 - ¿Cuál es la aplicación gratis con mayor ratio de reviews positivas? (⭐⭐)


Filtrar las aplicaciones gratuitas

In [ ]:
free_apps_rdd = google_play_data.filter(lambda x: x['Type'] == 'Free')

Filtrar las reseñas 'nan'

In [ ]:
filter_user_reviews_data = user_reviews_data.filter(lambda x: str(x['Sentiment']) != 'nan')


Cantidad de reseñas por app

In [ ]:
reviews_count_per_app = filter_user_reviews_data.map(lambda x: (x['App'], 1)).reduceByKey(lambda x, y: x + y)

Cantidad de reseñas positivas por app

In [ ]:
positive_reviews_count_per_app = filter_user_reviews_data.filter(lambda x: (x['Sentiment'] == 'Positive')).map(lambda x: (x['App'], 1)).reduceByKey(lambda x, y: x + y)


Calcular el ratio de reseñas positivas por aplicación

In [ ]:
ratio_reviews_per_app = positive_reviews_count_per_app.join(reviews_count_per_app).map(lambda x: (x[0], x[1][0] / x[1][1] ))


Encontrar la app con mayor ratio de reviews positivas

In [ ]:
ratio_reviews_per_app.reduce(lambda x, y: x if x[1] > y[1] else y)

('Home workouts - fat burning, abs, legs, arms,chest', 1.0)

#Como hay varias aplicaciones con ratio 1 (ratio maximo), propongo como solucion mostrar todas las apps que contengan el ratio maximo(sin hardcodear que el ratio maximo sea 1)


Encontrar el máximo valor de ratio

In [ ]:
max_ratio_value = ratio_reviews_per_app.map(lambda x: x[1]).reduce(lambda x, y: x if x > y else y)

Filtrar las aplicaciones con el máximo valor de ratio

In [ ]:
apps_with_max_ratio = ratio_reviews_per_app.filter(lambda x: x[1] == max_ratio_value)

In [ ]:
apps_with_max_ratio.collect()

[('3D Live Neon Weed Launcher', 1.0),
 ('Apartment Decorating Ideas', 1.0),
 ('Baby Panda’s Juice Shop', 1.0),
 ('Best Fiends - Free Puzzle Game', 1.0),
 ('CM Flashlight (Compass, SOS)', 1.0),
 ('Calculator Plus Free', 1.0),
 ('Caller ID +', 1.0),
 ('Daily Workouts - Exercise Fitness Routine Trainer', 1.0),
 ('DashClock Widget', 1.0),
 ('Down Dog: Great Yoga Anywhere', 1.0),
 ('Draw a Stickman: EPIC 2', 1.0),
 ('FlipaClip - Cartoon animation', 1.0),
 ('GPS Speedometer and Odometer', 1.0),
 ('Google Primer', 1.0),
 ('Google Translate', 1.0),
 ('Google+', 1.0),
 ('HomeWork', 1.0),
 ('7 Day Food Journal Challenge', 1.0),
 ('All-in-One Mahjong 3 FREE', 1.0),
 ('Amazon Prime Video', 1.0),
 ('Bed Time Fan - White Noise Sleep Sounds', 1.0),
 ('Best Ovulation Tracker Fertility Calendar App Glow', 1.0),
 ('Brightest Flashlight Free ®', 1.0),
 ('Caf - Mon Compte', 1.0),
 ('Calculator - unit converter', 1.0),
 ('Calendar+ Schedule Planner App', 1.0),
 ('CallApp: Caller ID, Blocker & Phone Call Re